# Notebook that generates my README

In [ ]:
import asyncio
import os
import warnings
from collections import Counter, defaultdict
from contextlib import asynccontextmanager
from datetime import datetime
import logging

import gidgethub.httpx
import httpx
import jinja2
import tenacity
import matplotlib.pyplot as plt
import matplotlib

logging.basicConfig(level=logging.INFO)

warnings.filterwarnings("ignore", message="findfont")

plt.xkcd()
matplotlib.rcParams.update({"font.family": ["Humor Sans"]})  # avoids findfont warnings


ME = "basnijholt"
orgs = (ME, "python-adaptive", "topocm", "python-kasa", "kwant-project")


def load_token():
    token_path = ".TOKEN"
    if os.path.exists(token_path):
        with open(token_path) as f:
            return f.read().strip()
    return os.environ["TOKEN"]


token = load_token()
retry_kw = dict(stop=tenacity.stop_after_attempt(10), wait=tenacity.wait_fixed(180))


@asynccontextmanager
async def gh_client(token):
    async with httpx.AsyncClient() as client:
        gh = gidgethub.httpx.GitHubAPI(client, ME, oauth_token=token)
        yield gh


@tenacity.retry(**retry_kw)
async def get_org_repos(org, token):
    repos = []
    async with gh_client(token) as gh:
        url = f"/users/{ME}/repos" if org == ME else f"/orgs/{org}/repos"
        async for repo in gh.getiter(f"{url}?type=sources"):
            repos.append(repo)
    return repos


async def get_all_repos_in_orgs(orgs, token):
    tasks = [get_org_repos(org, token) for org in orgs]
    all_repos = await asyncio.gather(*tasks)
    return sum(all_repos, [])


@tenacity.retry(**retry_kw)
async def get_repo(full_repo_name, token):
    async with gh_client(token) as gh:
        owner, name = full_repo_name.split("/")
        return await gh.getitem(f"/repos/{owner}/{name}")


async def get_repos(full_repo_names, token):
    tasks = [get_repo(full_repo_name, token) for full_repo_name in full_repo_names]
    return await asyncio.gather(*tasks)


@tenacity.retry(**retry_kw)
async def get_n_commits(full_repo_name, user=ME):
    async with gh_client(token) as gh:
        owner, name = full_repo_name.split("/")
        try:
            stats_contributors = await gh.getitem(
                f"/repos/{owner}/{name}/stats/contributors"
            )
        except Exception:
            print(f"Error: {full_repo_name}")
            return None

        if stats_contributors is None:
            print(f"API returned None for {full_repo_name}")
            return None

        total_commits = next(
            (s["total"] for s in stats_contributors if s["author"]["login"] == user),
            0,
        )
        return full_repo_name, total_commits


@tenacity.retry(**retry_kw)
async def get_stargazers_page_with_dates(gh, owner, name, page, headers):
    stats_contributors = await gh.getitem(
        f"/repos/{owner}/{name}/stargazers?per_page=100&page={page}",
        extra_headers=headers,
    )
    starred_at = [
        datetime.strptime(s["starred_at"], "%Y-%m-%dT%H:%M:%SZ")
        for s in stats_contributors
    ]

    return starred_at


@tenacity.retry(**retry_kw)
async def get_stargazers_with_dates(full_repo_name):
    headers = {"Accept": "application/vnd.github.v3.star+json"}
    starred = []
    async with gh_client(token) as gh:
        owner, name = full_repo_name.split("/")
        page = 1
        while True:
            logging.info(f"Fetching stargazers for {owner}/{name}, page {page}")
            starred_at = await get_stargazers_page_with_dates(
                gh, owner, name, page, headers
            )
            if not starred_at:
                break
            starred.extend(starred_at)
            page += 1
    return starred


@tenacity.retry(**retry_kw)
async def get_commits(full_repo_name, author=ME):
    async with gh_client(token) as gh:
        owner, name = full_repo_name.split("/")
        commits = []
        async for commit in gh.getiter(
            f"/repos/{owner}/{name}/commits?author={author}&per_page=100"
        ):
            commits.append(commit)
        return commits


def split(x, at_index=5):
    return x[:at_index], x[at_index:]


In [ ]:
repos = await get_all_repos_in_orgs(orgs, token)
full_repo_names = [repo["full_name"] for repo in repos]

In [ ]:
repos = await get_repos(full_repo_names, token)

## Number of stars ⭐️

In [ ]:
mapping = defaultdict(list)
for repo in repos:
    namespace, name = repo["full_name"].split("/", 1)
    mapping[namespace].append(repo)

most_stars = sorted(
    (repo for project in orgs for repo in mapping[project]),
    key=lambda r: r["stargazers_count"],
    reverse=True,
)

most_stars_strs = [
    f"{i+1}. [{repo['full_name']}](https://github.com/{repo['full_name']}/), {repo['stargazers_count']} ⭐️s"
    for i, repo in enumerate(most_stars[:20])
]

most_stars_strs_summary, most_stars_strs_rest = split(most_stars_strs, 5)

## Number of commits 

In [ ]:
to_check = []
for repo in repos:
    full_name = repo["full_name"] if not repo["fork"] else repo["source"]["full_name"]
    if full_name in ("regro/cf-graph-countyfair", "volumio/Volumio2"):
        # https://github.com/PyGithub/PyGithub/issues/1599
        continue
    to_check.append(full_name)

commits = await asyncio.gather(*[get_n_commits(full_name) for full_name in to_check])
commits = [c for c in commits if c is not None]

In [ ]:
most_committed = sorted(set(commits), key=lambda x: x[1], reverse=True)

most_committed_strs = [
    f"{i+1}. [{full_name}](https://github.com/{full_name}/), {n_commits} commits :octocat:"
    for i, (full_name, n_commits) in enumerate(most_committed[:20])
]

most_committed_strs_summary, most_committed_strs_rest = split(most_committed_strs, 5)

In [ ]:
stargazers = await asyncio.gather(
    *[get_stargazers_with_dates(r["full_name"]) for r in most_stars[:20]]
)
dts = sorted(sum(stargazers, []))
n_stars = list(range(1, len(dts) + 1))

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.set_xlabel("Year")
ax.set_ylabel("Cumulative number of stars")
ax.set_title("Total number of stars over time")

for r, stars in zip(most_stars, stargazers):
    n_stars = list(range(1, len(stars) + 1))
    ax.plot(stars, n_stars, label=r["full_name"])
plt.savefig("stars_over_time.png")

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(dts, n_stars)
ax.set_xlabel("Year")
ax.set_ylabel("Cumulative number of stars")
ax.set_title("Total number of stars over time")
plt.savefig("stars_over_time.png")

## Commit stats

In [ ]:
all_commits = await asyncio.gather(
    *[get_commits(full_name) for full_name, _ in most_committed[:5]]
)
all_commits = sum(all_commits, [])
all_commit_dates = [
    datetime.strptime(c["commit"]["author"]["date"], "%Y-%m-%dT%H:%M:%SZ")
    for c in all_commits
]

In [ ]:
weekdays = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]

day_hist = [
    (weekdays[i], n)
    for i, n in sorted(Counter([d.weekday() for d in all_commit_dates]).items())
]

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))
ax.bar(*zip(*day_hist))
ax.set_xlabel("Day of the week")
ax.set_ylabel("Total number of commits")
ax.set_title("Commits by day")
plt.savefig("commits_per_weekday.png")

In [ ]:
hour_hist = [
    (f"{i:02d}", n)
    for i, n in sorted(Counter([d.hour for d in all_commit_dates]).items())
]

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))
ax.bar(*zip(*hour_hist))
ax.set_xlabel("Hour of the day")
ax.set_ylabel("number of commits")
ax.set_title("Commits by hour")
plt.savefig("commits_per_hour.png")

## Render template

In [ ]:
template = """### Welcome to my profile 👋

<center>
  <table>
    <tr>
        <td><img width="300px" align="left" src="https://github-readme-stats.vercel.app/api/top-langs/?username=basnijholt&hide=TeX,Jupyter%20Notebook&layout=compact&theme=radical" /></td>
        <td><img align='right' src="https://github-readme-stats.vercel.app/api?username=basnijholt&show_icons=true&theme=radical" width="380"></td>
    </tr>
  </table>
</center>

![visitors](https://visitor-badge.glitch.me/badge?page_id=basnijholt.visitor-badge)

I am Bas. Here I present some (automatically generated) statistics about my activity on GitHub. For more info check out my website [www.nijho.lt](http://nijho.lt/).

![](https://www.nijho.lt/authors/admin/avatar_hu9e60e4b9bc120dfb6a666009f2878da6_182107_250x250_fill_q90_lanczos_center.jpg)

- 💬 Ask me about Python, home-automation, landscape photography, and quantum physics
- 📫 How to reach me: bas@nijho.lt

Last updated at {{ now }}.

# GitHub statistics — my top 20

## number of GitHub stars ⭐️

{{ "\n".join(most_stars_strs_summary) }}
<details><summary>Click to expand!</summary>

{{ "\n".join(most_stars_strs_rest) }}

</details>

![](https://github.com/basnijholt/basnijholt/raw/main/stars_over_time.png)

## number of commits :octocat:

{{ "\n".join(most_committed_strs_summary) }}
<details><summary>Click to expand!</summary>

{{ "\n".join(most_committed_strs_rest) }}

</details>

![](https://github.com/basnijholt/basnijholt/raw/main/commits_per_hour.png)

![](https://github.com/basnijholt/basnijholt/raw/main/commits_per_weekday.png)


"""
txt = jinja2.Template(template).render(
    most_stars_strs_summary=most_stars_strs_summary,
    most_stars_strs_rest=most_stars_strs_rest,
    most_committed_strs_summary=most_committed_strs_summary,
    most_committed_strs_rest=most_committed_strs_rest,
    now=str(datetime.now()),
)
with open("README.md", "w") as f:
    f.write(txt)
print(txt)